# ch 6 학습 관련 기술들

## 6.1 매개변수 갱신
신경망 학습의 목적: 손실 함수의 값을 가능한 한 낮추는 매개변수를 찾는 것; 매개변수의 '최적값'을 찾는 것--> 최적화

확률적 경사 하강법(SGD): 매개변수의 기울기(미분)을 이용하여 매개변수 값을 갱신하는 일을 반복하여 최적의 값에 다가가는 것

SGD의 단점: 비효율일 때가 있음; 비등방성(방향에 따라 성질(기울기)가 달라지는 함수)에서는 탐색 경로가 비효율적이다.

--> 모멘텀, AdaGrad, Adam

### 6.1.1 모멘텀
모멘텀(momenetum) : 운동량을 뜻하는 단어.

모멘텀의 갱신 경로는 공이 그릇 바닥을 구르듯 움직인다. SGD와 비교하면 지그재그 정도가 덜하다.  
--> x축의 힘은 아주 작지만 방향은 변하지 않아서 한 방향으로 일정하게 가속. y축의 힘은 크지만 위아래로 번갈아 받아서 상충하여 y축 방향의 속도는 안정적이지 않다.

전체적으로 SGD보다 x축 방향으로 빠르게 다가가 지그재그 움직임이 줄어든다.

In [1]:
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None #물체의 속도 처음에는 아무것도 담지 않는다.
        
    def update(self, params, grads):
        if self.v is None: #update가 처음 호출도리 때, 매개변수와 같은 구조의 데이터를 딕셔너리 변수로 저장.
            self.v = {}
            for key, val in params.items():                                
                self.v[key] = np.zeros_like(val)
                
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key] 
            params[key] += self.v[key]

### 6.1.2 AdaGrad
AdaGrad은 각각의 매개변수에 '맞춤형' 값을 만들어준다; 개별 배개변수에 적응적으로 학습률을 조정하면서 학습 진행.
과거의 기울기를 제곱하여 계속 더해간다; 학습을 진행할수록 갱신 강도가 약해진다. (무한히 계속 학습하면 갱신량이 0이 되어 갱신하지 않게 됨)

신경망 학습에서는 학습률의 값이 너무 작으면 학습 시간이 길어지고, 너무 크면 발산하여 학습이 제대로 이루어지지 않기 때문에 학습률의 값이 중요하다. 이 학습률을 정하는 효과적 기술로 학습률 감소가 있다. 학습을 진행하면서 점차 학습률을 줄여가는 방법. 처음에는 크게학습하다가 조금씩 작게 학습하는 것.



In [2]:
class AdaGrad:

    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
            
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7) #1e-7 를 더해주는 것!!!
            # 1e-7를 더해줌으로서 0으로 나누는 사태를 막아준다.

### 6.1.3 Adam
Adam : 모멘텀과 AdaGrad 기법을 융합한 것
매개변수 공간을 효율적으로 탐색, 하이퍼파라미터의 '편향 보정'이 진행된다.

## 6.2 가중치의 초깃값

가중치의 초깃값을 설정하는 것은 중요. 
가중치 감소: 오버피팅을 억제해 범용 성능을 높이는 테크닉. 가중치 매개변수의 값이 작아지도록 학습하는 방법. 가중치 값을 작게하여 오버피팅이 일어나지 않게 하는 것. 

but, 가중치의 값을 0으로 하면 안된다; 오차전역전파법에서 모든 가중치의 값이 또같이 갱신되기 때문.

활성화 함수로 ReLU 사용할 때는 He 초깃값을, sigmoid 나 tanh 등의 S자 모양 곡선일 때는 Xavier 초깃값 사용.

## 6.3 배치 정규화

각 층이 활성화를 적당히 퍼뜨리도록 강제한다면? --> '배치 정규화'가 이런 아이디어에서 출발한 방법.

배치 정규화가 주목받는 이유
1. 학습을 속도 개선
2. 초긱값에 크게 의존하지 않는다
3. 오버피팅을 억제한다

## 6.4 바른 학습을 위해

오버피팅: 신경망이 훈련 데이터에만 지나치게 적용되어 그 외의 데이터에는 제대로 대응하지 못하는 상태.
오버피팅을 억제하는 기술이 중요하다.

오버피팅은 아래 경우에 발생
1. 매개변수가 많고 표현력이 높은 모델
2. 훈련 데이터가 적음

### 6.4.1 가중치 감소

가중치 감소 for 오버피팅 억제; 학습 과정에서 큰 가중치에 대해 큰 페널티를 부과.
(원래 오버피팅은 가중치 매개변수의 값이 커서 발생하는 경우가 많기 때문)

### 6.4.2 드롭아웃

드롭아웃 for 오버피팅 억제: 뉴런을 임의로 삭제하면서 학습하는 방법.
뉴런을 무작위로 골라 삭제, 삭제된 뉴런은 신호를 전달하지 ㅇ낳는다. 

In [3]:
class Dropout:
    def __init__(self, dropout_ratio = 0.5):
        self.dropout_ratio = dropout_ratio
        self.mask = None
        
    def forward(self, x, train_flag=True):
        if train_flag:
            #x와 형상이 같은 배열을 무작위로 생성, 그 값이 ratio 보다 큰 원소만 true로 설정.
            self.mask = np.random.rand(*x.shape)>self.dropout_ratio 
            return x*self.mask
        else:
            return x * (1.0 - self.dropout_ratio)
        
    def backward(self, dout):
        return dout * self.mask

## 6.5 하이퍼파라미터

최적화의 핵심: '최적값'이 존재하는 범위를 조금씩 줄여나가는 것.
대략적인 범위 설정 -> 무작위로 하이퍼파라미터 값을 샘플링 -> 이 값으로 정확도 평가.

순서:
1. 하이퍼파라미터 값의 범위 설정
2. 설정된 범위에서 하이퍼파라미터의 값을 무작위로 추출
3. 1단계에서 샘플링한 하이퍼파라미터 값을 사용하여 학습, 검증 데이터로 정확도 평가
4. 2단계와 3단계를 특정 횟수 반복, 정확도의 결과를 보고 하이퍼파라미터의 범위를 좁힌다.

## 6.6 정리
1. 매개변수 갱신 방법에는 SGD외에도 모멘텀, AdaGrad, Adam 등이 있다
2. 가중치 초깃값을 정하는 방법은 매우 중요
3. 가중치 초깃값으로는 'Xavier 초깃값'과 'He 초깃값'이 효과적
4. 배치 정규화 이용하면 학습을 빠르게 진행 가능, 또한 초깃값에 영향을 덜 받음
5. 오버피팅 억제: 가중치 감소, 드롭아웃
6. 하이퍼파라미터 값 탐색은 최적 값이 존재할 법한 범위를 점차 좁히면서 하는 것이 효과적.